In [ ]:
!pip install -qq statsmodels seaborn pymc ipywidgets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import beta
from statsmodels.stats.proportion import proportions_ztest

### The law of large numbers

In [ ]:
n = 1000
timesseries = []
for _ in range(10):
    values = np.random.randint(0, 2, (n))
    index = np.arange(n) + 1
    cum = np.cumsum(values)
    pcts = cum / index
    timesseries.append(pcts)

plt.figure(figsize=(13, 4))
for series in timesseries:
    plt.plot(series, alpha=0.7)
plt.axhline(0.5, c="black", ls="--")
plt.axvline(10, c="black", ls="--")
plt.axvline(100, c="black", ls="--")
plt.show()

### A/B tests (counts)

In [ ]:
control_nobs = 100
control_positive = 15
control_rate = control_positive / control_nobs
treatment_nobs = 100
treatment_positive = 22
treatment_rate = treatment_positive / treatment_nobs
print(control_rate, treatment_rate)

frequentist

In [ ]:
def rate_test(control_positive, treatment_positive, control_nobs, treatment_nobs):
    count = [control_positive, treatment_positive]
    nobs = [control_nobs, treatment_nobs]
    count, nobs = map(np.array, [count, nobs])
    stat, pvalue = proportions_ztest(count, nobs)
    print(f"stat={stat:.3f}, pvalue={pvalue:.3f}")
    return stat, pvalue


stat, pvalue = rate_test(
    control_positive, treatment_positive, control_nobs, treatment_nobs
)
assert stat + 1.275 < 1e-3
assert pvalue - 0.202 < 1e-3

baysian

In [ ]:
def baysian_ab_test(
    control_positive: int,
    treatment_positive: int,
    control_nobs: int,
    treatment_nobs: int,
    a_prior: int = 1,
    b_prior: int = 1,
    n_sims: int = 1_000_000,
):
    a_control = a_prior + control_positive
    b_control = b_prior + (control_nobs - control_positive)
    a_treatment = a_prior + treatment_positive
    b_treatment = b_prior + (treatment_nobs - treatment_positive)

    control_posterior = beta.rvs(a_control, b_control, size=n_sims)
    treatment_posterior = beta.rvs(a_treatment, b_treatment, size=n_sims)

    prob_better = np.mean(control_posterior < treatment_posterior)
    lift = np.mean(treatment_posterior - control_posterior)
    prob_of_lift_gt_x = {}
    for i in [0.01, 0.02, 0.03, 0.04, 0.05]:
        prob_of_lift_gt_x[i] = np.mean(treatment_posterior - control_posterior > i)
    pct2p5, pct97p5 = np.percentile(treatment_posterior, [2.5, 97.5])

    print(f"Prob treatment gt control: {prob_better:.2f}")
    print(f"Average lift: {lift:.2f}")
    print(f"CI 5%: {pct2p5:.2f} - {pct97p5:.2f}")
    print("Prob of lift greater than:")
    for k, v in prob_of_lift_gt_x.items():
        print(f"\t{k}: {v:.2f}")

    plt.figure()
    plt.hist(control_posterior, bins=50, label="control", alpha=0.7)
    plt.hist(treatment_posterior, bins=50, label="treatment", alpha=0.7)
    plt.xlim(0, 1)
    plt.legend()
    plt.show()

    return prob_better, lift


prob_better, lift = baysian_ab_test(
    control_positive, treatment_positive, control_nobs, treatment_nobs
)
assert prob_better - 0.9 < 1e-3
assert lift - 0.07 < 1e-3

### A/B tests (continious)

In [ ]:
import pymc as pm

n = 100

treatment = np.random.normal(loc=11, scale=3, size=n)
control = np.random.normal(loc=10, scale=2, size=n)

with pm.Model() as model:
    mu1 = pm.Normal("mu1", mu=0, sigma=10)
    mu2 = pm.Normal("mu2", mu=0, sigma=10)

    sigma1 = pm.HalfNormal("sigma1", sigma=10)
    sigma2 = pm.HalfNormal("sigma2", sigma=10)

    y1 = pm.Normal("y1", mu=mu1, sigma=sigma1, observed=treatment)
    y2 = pm.Normal("y2", mu=mu2, sigma=sigma2, observed=control)

    diff = pm.Deterministic("diff", mu1 - mu2)

    trace = pm.sample()
samples = trace.posterior["diff"].values.flatten()

prob_better = np.mean(samples > 0)
lift = np.mean(samples)
lb, ub = np.percentile(trace.posterior["mu1"].values.flatten(), [0.05, 0.95])

plt.figure()
plt.hist(treatment, label="treatment", alpha=0.7)
plt.hist(control, label="control", alpha=0.7)
plt.legend()
plt.show()

print(f"Prob treatment gt control: {prob_better:.2f}")
print(f"Average lift: {lift:.2f}")

plt.hist(samples, bins=50)
plt.axvline(0, color="r", linestyle="--")
plt.title("Posterior samples of difference")
plt.show()

treatment_samples = trace.posterior["mu1"].values.flatten()
control_samples = trace.posterior["mu2"].values.flatten()

plt.figure(figsize=(10, 6))
plt.hist(treatment_samples, bins=50, alpha=0.5, label="mu1 posterior")
plt.hist(control_samples, bins=50, alpha=0.5, label="mu2 posterior")
plt.axvline(treatment.mean(), color="blue", linestyle="--", label="grp1 sample mean")
plt.axvline(control.mean(), color="orange", linestyle="--", label="grp2 sample mean")
plt.title("Posterior Distributions of mu1 and mu2")
plt.legend()
plt.show()

# Print some stats
print(
    f"treatment: mean={np.mean(treatment_samples):.3f}, "
    f"std={np.std(treatment_samples):.3f}"
)
print(
    f"control: mean={np.mean(control_samples):.3f}, std={np.std(control_samples):.3f}"
)

### Bayesian bootstrap

Can be used for baysian AB test with non-normal continious data

In [ ]:
n_samples: int = 10_000

treatment = np.concatenate([np.random.normal(11, 2, 50), np.random.exponential(2, 50)])
control = np.concatenate([np.random.normal(10, 2, 50), np.random.exponential(2, 50)])

weights_treatment = np.random.dirichlet([1] * len(treatment), n_samples)
weights_control = np.random.dirichlet([1] * len(control), n_samples)


means_treatment = np.dot(weights_treatment, treatment)
means_control = np.dot(weights_control, control)

diff = means_treatment - means_control
prob_better = np.mean(diff > 0)
lift = np.mean(diff)
ci_lower, ci_upper = np.percentile(diff, [2.5, 97.5])


# plot raw data
plt.figure()
plt.hist(treatment, alpha=0.7, label="treatment", bins=30)
plt.hist(control, alpha=0.7, label="control", bins=30)
plt.legend()
plt.show()


# Print results
print(f"Probability that treatment is better than control: {prob_better:.1%}")
print(f"Lift: {lift:.2f}")
print(f"95% Credible Interval: ({ci_lower:.2f}, {ci_upper:.2f})")

# Plot the bootstrap distributions
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(means_treatment, bins=50, alpha=0.5, label="treatment")
plt.hist(means_control, bins=50, alpha=0.5, label="control")
plt.xlabel("Mean Value")
plt.ylabel("Frequency")
plt.title("Bootstrap Distribution of Means")
plt.legend()

# Plot the difference distribution
plt.subplot(1, 2, 2)
plt.hist(diff, bins=50)
plt.axvline(0, color="red", linestyle="--")
plt.ylabel("Frequency")
plt.title("Distribution of Differences")
plt.tight_layout()
plt.show()

### multinomial ab test

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data: counts of observations in each category
observed_counts = np.array([10, 5, 3])  # three categories
num_categories = len(observed_counts)

# Prior parameters (concentration parameters)
# Using α = 1 for each category (uniform prior)
alpha_prior = np.ones(num_categories)

# Posterior parameters
alpha_posterior = alpha_prior + observed_counts

# Generate samples from the posterior distribution
num_samples = 10000
posterior_samples = np.random.dirichlet(alpha_posterior, num_samples)

# Calculate expected probabilities
expected_probs = alpha_posterior / alpha_posterior.sum()

print("Expected probabilities:", expected_probs)

# Visualize the distributions for each category
plt.figure(figsize=(12, 6))
for i in range(num_categories):
    plt.hist(posterior_samples[:, i], bins=50, alpha=0.5, label=f"Category {i+1}")
    plt.axvline(expected_probs[i], color=f"C{i}", linestyle="--")

plt.title("Posterior Distributions of Category Probabilities")
plt.xlabel("Probability")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Generate a single sample of probabilities
sample_probs = np.random.dirichlet(alpha_posterior)
print("\nSample probabilities:", sample_probs)

# Simulate new data using these probabilities
new_sample_size = 100
new_data = np.random.multinomial(new_sample_size, sample_probs)
print("\nSimulated new counts:", new_data)

# Calculate 95% credible intervals for each category
credible_intervals = np.percentile(posterior_samples, [2.5, 97.5], axis=0)
print("\n95% Credible Intervals:")
for i in range(num_categories):
    print(
        f"Category {i+1}: ({credible_intervals[0,i]:.3f}, "
        f"{credible_intervals[1,i]:.3f})"
    )

In [ ]:
np.mean(posterior_samples[0] > posterior_samples[-1])